### Image classification test based on basic neural network

pp. 58

1. Read the data: Transform the data into tensors, the input shape for the network.
2. Define the network (using Multi Layer Perceptron)
3. Define loss function
4. Define optimization
5. Train the network
6. Make a prediction

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# 1. READ THE DATA: Download the dataset "CIFAR-10"

train_dataset = torchvision.datasets.CIFAR10(root='../CIFAR10', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.CIFAR10(root='../CIFAR10', train=False, transform=transforms.ToTensor(), download=True)

image, label = train_dataset[0]
print(image.size())
print(label)

Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False, num_workers=2)

In [5]:
for images, labels in train_loader:
    print(images.size()) # loaded tensors
    print(images[0].size()) # first batch
    print(labels.size())
    break

torch.Size([64, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([64])


In [6]:
# 2. DEFINE THE NETWORK: based on MLP

num_classes = 10
class MLPNet (nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 600) # fully connected layer 1
        self.fc2 = nn.Linear(600, 600)     # fully connected layer 2
        self.fc3 = nn.Linear(600, num_classes) # fully connected layer 3
        self.dropout1 = nn.Dropout2d(0.2) # droptout layer 1
        self.dropout2 = nn.Dropout2d(0.2) # dropout layer 2
        
    def foward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        return F.relu(self.fc3(x))
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = MLPNet().to(device)

In [7]:
# 3 & 4. DEFINE LOSS FUNCTION AND OPTIMIZATION

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

In [ ]:
# 5. TRAIN THE NETWORK

num_epochs = 50

train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

for epoch in range(num_epochs):
    train_loss = 0
    train_acc = 0
    val_loss = 0
    val_acc = 0
    
    ##### train #####
    net.train()
    for i (images, labels) in enumerate(train_loader):
        images, labels = images.view(-1, 32*32*3).to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        train_loss +=loss.item()
        train_acc += (outputs.max(1)[1]==labels).sum().item()
        loss.backward()
        optimizer.step()
        
    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_acc = train_acc / len(train_loader.dataset)
    
    ##### evaluation #####
    net.eval()